Hmm I should be able to calculate the truth from any label I use right? Let's check that!!


In [13]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
plt.style.use(['seaborn-v0_8', 'seaborn-v0_8-paper'])
# plt.style.use("ggplot")

import datasets
from datasets import Dataset
from sklearn.metrics import roc_auc_score


In [2]:
# load my code
%load_ext autoreload
%autoreload 2

from src.datasets.dm import DeceptionDataModule


In [3]:

import seaborn as sns


MAX_SAMPLES = 1000


In [7]:
f1_ood = '/media/wassname/SGIronWolf/projects5/elk/sgd_probes_are_lie_detectors/.ds/ds_OOD_6d3ece46c44f6c3b'
f1_val = '/media/wassname/SGIronWolf/projects5/elk/sgd_probes_are_lie_detectors/.ds/ds_valtest_73b754e8fdff9f2f'

ds_val = Dataset.from_file(f1_val).with_format("torch")

ds_oos = Dataset.from_file(f1_ood).with_format("torch")

ds_out1 = datasets.interleave_datasets([ds_val, ds_val])
ds_out = ds_out1.select(range(MAX_SAMPLES))
len(ds_out1), ds_out


/media/wassname/SGIronWolf/projects5/elk/sgd_probes_are_lie_detectors/.venv/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


(3204,
 Dataset({
     features: ['end_logits_base', 'choice_probs_base', 'binary_ans_base', 'label_true_base', 'label_instructed_base', 'instructed_to_lie_base', 'sys_instr_name_base', 'example_i_base', 'ds_string_base', 'template_name_base', 'correct_truth_telling_base', 'correct_instruction_following_base', 'end_residual_stream_base', 'end_logits_adapt', 'choice_probs_adapt', 'binary_ans_adapt', 'label_true_adapt', 'label_instructed_adapt', 'instructed_to_lie_adapt', 'sys_instr_name_adapt', 'example_i_adapt', 'ds_string_adapt', 'template_name_adapt', 'correct_truth_telling_adapt', 'correct_instruction_following_adapt', 'end_residual_stream_adapt'],
     num_rows: 1000
 }))

In [8]:
ds = ds_out


# Check

In [23]:
from src.eval.labels import ranking_truth_telling, ranking_instruction_following


In [20]:
y_true = ds['label_true_base']


In [21]:
# binary_ans is just the binary prediction
score = roc_auc_score(y_true, ds['binary_ans_base'])
print(f"Base model score: {score:.2%}")

score = roc_auc_score(y_true, ds['binary_ans_adapt'])
print(f"Adapt model score: {score:.2%}")


Base model score: 80.72%
Adapt model score: 90.98%


## ranking_truth_telling

In [85]:
from src.eval.labels import ranking_truth_telling, undo_ranked_truth_telling


# UNIT TEST
y_true = ds['label_true_base']
base_more_truthfull = ranking_truth_telling(ds) * 0.9
y_true_from_rank = undo_ranked_truth_telling(ds, base_more_truthfull)>0.5

np.testing.assert_array_equal(y_true, y_true_from_rank)


## ranking_instruction_following

In [ ]:
ranking_instruction_following


In [5]:

df_res_ab_o, df_res_o
ds = ds_out_OOD
a = ds2label_model_truth(ds).numpy()
t = ds['label_true_base']

# a = ds2label_model_obey(ds_out_OOD)
b = (ds['binary_ans_base'].numpy()>0.5) ^ (ds['correct_truth_telling_base']>0.5).numpy()

c = ranking_truth_telling(ds).numpy()
c = c * ds['correct_truth_telling_adapt'].numpy()

# c = ranking_instruction_following(ds_out_OOD)
# ds_out_OOD['correct_truth_telling_adapt']
# ds_out_OOD['correct_instruction_following_base']
# ds_out_OOD['correct_instruction_following_adapt']
# plt.scatter(a, b, c=c, cmap='RdYlGn')
# plt.show()
# plt.scatter(a, c, c=b, cmap='RdYlGn')

df_labels = pd.DataFrame([t, a, b, c], index=['label', 'model_true', 'correct_truth_telling_base', 'ranking_truth_telling']).T
df_labels.corr()


NameError: name 'df_res_ab_o' is not defined

# Jointplot?

In [ ]:
# # https://github.com/lingo-mit/lm-truthfulness/blob/master/lm_truthfulness_gpt-j.ipynb
# import seaborn as sns
# g = sns.JointGrid(data=df, x="lm", y="probe", hue="type", xlim=(0, 1), ylim=(0, 1), space=0, palette=palette)
# g.plot_joint(
#     sns.histplot, #discrete=(True, False),
#     binrange=[0, 1],
#     bins=20,
#     #cmap="gist_gray_r",
# )
# g.plot_marginals(
#     sns.histplot, multiple="stack",
#     binrange=[0, 1],
#     bins=20,
#     #color="gray",

# )
# g.figure.axes[0].set_xticks([0, 0.5, 1])
# g.figure.axes[0].set_yticks([0, 0.5, 1])
# g.figure.axes[0].set_axisbelow(False)
# g.figure.axes[0].grid(color="black")
# g.figure.axes[0].get_legend().remove()
# #g.plot_joint(sns.scatterplot, sns.histplot)
# #g = sns.jointplot(data=df, x="lm", y="probe")
# plt.show()

